In [25]:
import pandas as pd
import jupyter_black

In [26]:
all_games_df = pd.read_parquet('../data/standings/dodgers_yankees_history_1924_2024.parquet')

In [27]:
dodgers_df = all_games_df.query('team == "LAD" or team == "BRO"').reset_index(drop=True).copy()

In [28]:
yankees_dodgers_df = dodgers_df.query('opp == "NYY"').reset_index(drop=True).copy()

In [185]:
reg_season_games = len(yankees_dodgers_df)

---

In [151]:
postseason_table = pd.read_html('https://www.baseball-reference.com/postseason/')[0]
postseason_df = postseason_table.rename(columns={'Series': 'series', 'Unnamed: 1': 'split', 'Unnamed: 2': 'teams'})
postseason_df[['winner', 'loser']] = postseason_df['teams'].str.split(' vs. ', expand=True)

In [152]:
postseason_df['w_name'] = postseason_df['winner'].str.replace(')', '').str.split('(', expand=True)[0]
postseason_df['w_record'] = postseason_df['winner'].str.replace(')', '').str.split('(', expand=True)[1]
postseason_df['w_league'] = postseason_df['w_record'].str.split(',', expand=True)[1]

postseason_df['l_name'] = postseason_df['loser'].str.replace(')', '').str.split('(', expand=True)[0]
postseason_df['l_record'] = postseason_df['loser'].str.replace(')', '').str.split('(', expand=True)[1]
postseason_df['l_league'] = postseason_df['l_record'].str.split(',', expand=True)[1]

In [153]:
postseason_df['w_games'] = postseason_df['split'].str.split('-', expand=True, n=1)[0]
postseason_df['l_games'] = postseason_df['split'].str.split('-', expand=True, n=1)[1]

In [154]:
postseason_slim_df = postseason_df.drop(['teams', 'winner', 'loser', 'w_record', 'l_record', 'split'], axis=1).dropna().reset_index(drop=True).copy()

In [155]:
postseason_slim_df['year'] = postseason_slim_df['series'].str.split(' ', n=1, expand=True)[0]
postseason_slim_df['series'] = postseason_slim_df['series'].str.split(' ', n=1, expand=True)[1]

In [158]:
world_series_df =  postseason_slim_df.query('series.str.contains("World Series")')

In [159]:
# Filter the DataFrame to show only rows where the winning or losing team contains "Yankees" or "Dodgers"
filtered_df = world_series_df[
    world_series_df['w_name'].str.contains("Yankees|Dodgers", case=False) |
    world_series_df['l_name'].str.contains("Yankees|Dodgers", case=False)
]

In [ ]:
# Filter the DataFrame to show only rows where the Yankees played against the Dodgers
yankees_vs_dodgers_df = world_series_df[
    (world_series_df['w_name'].str.contains("Yankees", case=False) & world_series_df['l_name'].str.contains("Dodgers", case=False)) |
    (world_series_df['w_name'].str.contains("Dodgers", case=False) & world_series_df['l_name'].str.contains("Yankees", case=False))
]

In [210]:
yankees_vs_dodgers_df

,series,w_name,w_league,l_name,l_league,w_games,l_games,year
283,World Series,Los Angeles Dodgers,NL,New York Yankees,AL,4,2,1981
296,World Series,New York Yankees,AL,Los Angeles Dodgers,NL,4,2,1978
299,World Series,New York Yankees,AL,Los Angeles Dodgers,NL,4,2,1977
331,World Series,Los Angeles Dodgers,NL,New York Yankees,AL,4,0,1963
338,World Series,New York Yankees,AL,Brooklyn Dodgers,NL,4,3,1956
339,World Series,Brooklyn Dodgers,NL,New York Yankees,AL,4,3,1955
341,World Series,New York Yankees,AL,Brooklyn Dodgers,NL,4,2,1953
342,World Series,New York Yankees,AL,Brooklyn Dodgers,NL,4,3,1952
345,World Series,New York Yankees,AL,Brooklyn Dodgers,NL,4,1,1949
350,World Series,New York Yankees,AL,Brooklyn Dodgers,NL,4,3,1947


In [ ]:
ws_together_count = len(yankees_vs_dodgers_df)

In [193]:
last_ws_year = yankees_vs_dodgers_df['year'].max()
first_ws_year = yankees_vs_dodgers_df['year'].min()

In [167]:
dodgers_wins_df = yankees_vs_dodgers_df.query('w_name.str.contains("Dodgers")')
yankees_wins_df = yankees_vs_dodgers_df.query('w_name.str.contains("Yankees")')

In [170]:
dodgers_wins = len(dodgers_wins_df)
yankees_wins = len(yankees_wins_df)

In [184]:
dodgers_games_won = dodgers_wins_df['w_games'].astype(int).sum()
dodgers_games_won

12

In [ ]:
yankees_games_won = yankees_wins_df['w_games'].astype(int).sum()
yankees_games_won

In [194]:
ws_games_count = yankees_games_won + dodgers_games_won

In [187]:
total_games_played = yankees_games_won + dodgers_games_won + reg_season_games
total_games_played

66

In [209]:
f"Baseball games between the Los Angeles Dodgers and New York Yankees are relatively rare. Historically, the teams have been in different Major League Baseball leagues, leading to only {total_games_played} matchups over the past 100 years, most of which were in the World Series. They have faced off in the championship {ws_together_count} times, starting in {first_ws_year}, with the most recent encounter in {last_ws_year}. The Yankees have dominated, winning {yankees_wins} of those series, while the Dodgers have claimed victory in {dodgers_wins}. Since the introduction of interleague play in 1997, the teams have also met {reg_season_games} times outside the World Series."

'Baseball games between the Los Angeles Dodgers and New York Yankees are relatively rare. Historically, the teams have been in different Major League Baseball leagues, leading to only 66 matchups over the past 100 years, most of which were in the World Series. They have faced off in the championship 11 times, starting in 1941, with the most recent encounter in 1981. The Yankees have dominated, winning 8 of those series, while the Dodgers have claimed victory in 3. Since the introduction of interleague play in 1997, the teams have also met 22 times outside the World Series.'